In [98]:
import duckdb
import pandas as pd
import geopandas as gpd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import statsmodels.api as sm
from obsq.utils.core import convert_df_to_gdf
path = "/home/manat/projects/obsq/work/dev/data/data.duckdb"
target = 'expert_match'

In [99]:
table = 'observer'
schema = 'features'
con = duckdb.connect(path)
df = con.execute(f"""SELECT* FROM {schema}.{table}""" ).df()
#df = df.drop(columns= 'geom')
con.close()

In [100]:
df.shape

(2280, 29)

In [101]:
def plot_cat_rate(df, cat_col, target = target):
    output = "/home/manat/projects/obsq/work/dev/data/eda/preprocessed"
    rates = (
        df.groupby(cat_col)[target]
        .mean()
        .sort_values()
    )
    sns.barplot(x=rates.index[:20], y=rates.values[:20])
    plt.xticks(rotation=45, ha='right')   # or rotation=60, ha='right'
    plt.title(f"Observations rates for {cat_col}", fontsize=16)
    plt.savefig(f"{output}/{cat_col}_target_rates.jpg", dpi = 300, bbox_inches="tight")


In [102]:
def plot_logistic_reg(df, x_col:str):

    # Fit logistic regression
    X = sm.add_constant(df[x_col])
    model = sm.Logit(df[target], X).fit(disp=False)
    # Make smooth x grid for curve
    xx = np.linspace(df[x_col].min(), df[x_col].max(), 200)
    XX = sm.add_constant(xx)
    yy = model.predict(XX)
    # Plot
    sns.scatterplot(data=df, x=x_col, y=target, alpha=0.2)
    plt.plot(xx, yy, linewidth=2)
    plt.ylabel("P(y=1 | x)")
    plt.show()

In [103]:
def plot_mean_response(df, x):
    df.groupby(x)[target].mean().plot(kind="bar")
    plt.ylabel("P(y=1 | x)")
    plt.xlabel("x")
    plt.show()

In [104]:
def plot_boxplot(df,x_col):
    sns.boxplot(data=df, x=target, y=x_col)

In [105]:
def reduce_categories(df, col, k=20):
    top = df[col].value_counts().nlargest(k).index
    df[col + "_top"] = df[col].where(df[col].isin(top), "Other")
    return df


In [106]:
df.describe()

,observations_count,total_pct,expert_ids,expert_ids_pct,expert_match_count,expert_match_pct,avg_id_time,class_count,order_count,family_count,...,high_cood_un_obs,high_cood_un_pct,avg_coord_un,avg_media_count,sex_meta_pct,reproductiveCondition_meta_pct,annotations_meta_pct,avg_description_len,avg_species_obs_count,max_species_obs_count
count,2280.000000,2280.000000,2280.000000,2280.000000,2280.000000,2280.000000,2280.000000,2280.000000,2280.000000,2280.000000,...,2280.000000,2280.000000,2280.000000,2280.000000,2280.000000,2280.000000,2280.000000,2280.000000,2280.000000,2280.000000
mean,17.435965,0.000441,0.827193,0.081927,0.207895,0.043889,32.442544,1.835965,5.160088,6.838596,...,0.356140,0.047257,155.051035,14.329491,0.007378,0.107499,0.037505,4.669737,1.150974,1.829386
std,95.711847,0.002407,3.992968,0.215408,0.405890,0.155639,102.279016,1.066596,6.542753,11.490130,...,1.399996,0.171280,380.310347,9.051353,0.064094,0.235492,0.136973,21.003336,1.973951,6.181496
min,1.000000,0.000030,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000
25%,1.000000,0.000030,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,...,0.000000,0.000000,6.000000,10.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000
50%,3.000000,0.000080,0.000000,0.000000,0.000000,0.000000,1.500000,1.000000,2.000000,2.000000,...,0.000000,0.000000,18.410500,10.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000
75%,8.000000,0.000200,1.000000,0.010250,0.000000,0.000000,16.000000,2.000000,6.000000,7.000000,...,0.000000,0.000000,92.250000,15.710000,0.000000,0.091000,0.000000,0.000000,1.000000,1.000000
max,3389.000000,0.085250,88.000000,1.000000,1.000000,1.000000,1455.000000,5.000000,39.000000,97.000000,...,19.000000,1.000000,2946.000000,112.000000,1.000000,1.000000,1.000000,294.000000,92.000000,198.000000


In [107]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2280 entries, 0 to 2279
Data columns (total 29 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   recordedBy                      2280 non-null   object 
 1   observations_count              2280 non-null   int64  
 2   total_pct                       2280 non-null   float64
 3   expert_ids                      2280 non-null   int64  
 4   expert_ids_pct                  2280 non-null   float64
 5   expert_match_count              2280 non-null   int64  
 6   expert_match_pct                2280 non-null   float64
 7   avg_id_time                     2280 non-null   float64
 8   class_count                     2280 non-null   int64  
 9   order_count                     2280 non-null   int64  
 10  family_count                    2280 non-null   int64  
 11  genus_count                     2280 non-null   int64  
 12  species_count                   22

In [108]:
missing_values_count = df.isnull().sum()
missing_values_count

recordedBy                        0
observations_count                0
total_pct                         0
expert_ids                        0
expert_ids_pct                    0
expert_match_count                0
expert_match_pct                  0
avg_id_time                       0
class_count                       0
order_count                       0
family_count                      0
genus_count                       0
species_count                     0
unique_year_count                 0
unique_dates                      0
max_yearly_observations           0
max_monthly_observations          0
avg_yearly_observations           0
avg_monthly_observations          0
high_cood_un_obs                  0
high_cood_un_pct                  0
avg_coord_un                      0
avg_media_count                   0
sex_meta_pct                      0
reproductiveCondition_meta_pct    0
annotations_meta_pct              0
avg_description_len               0
avg_species_obs_count       